## Dependencias

In [43]:
# Basic data manipulation
import numpy as np
import pandas as pd
from datetime import datetime
from glob import glob
from uuid import uuid4
from sqlalchemy.engine import create_engine
from sqlalchemy.types import VARCHAR,FLOAT,INTEGER,UUID,DATETIME,CHAR
import json 

## Listar archivos

In [18]:
ruta = '../ibm_card_txn/*.csv'
archivos = np.random.choice(glob(ruta), 20)

## Limpieza de datos 

In [19]:
def validar_datos(archivo:str)->pd.DataFrame:
    """
    Esta función recibe la ruta de un archivo CSV, lee columnas definidas,
    convierte ciertos campos a valores numéricos y fechas en formato datetime,
    estandariza los nombres de columnas y devuelve un DataFrame resultante
    con la información validada.
    """
    cols = ['User', 'Card', 'Year', 'Month', 'Day', 'Time', 'Amount', 
            'Use Chip','Merchant State', 'MCC', 'Errors?', 'Is Fraud?']
    origin = ['User', 'Card', 'Amount', 
              'Use Chip','Merchant State', 'MCC', 'Errors?', 'Is Fraud?','ts']
    names = ['id_user','id_card','c_amt','d_use_chip','d_merchant_state',
             'id_mcc','d_errors','b_fraud','dt_timestamp']

    df = pd.read_csv(archivo, dtype=str, usecols=cols)

    for c in ['User','Card','MCC']:
        df[c] = pd.to_numeric(df[c], errors='coerce').fillna(99999).astype(int)
    
    for c in ['Month','Day']:
        df[c] = pd.to_numeric(df[c], errors='coerce').map(lambda x:f'{x:02d}')
    
    df['ts'] = df[['Year','Month','Day']].apply("-".join,axis=1)
    df['ts'] = df[['ts','Time']].apply(" ".join,axis=1)
    df['ts'] = pd.to_datetime(df['ts'],format='%Y-%m-%d %H:%M')

    df['Amount'] = pd.to_numeric(df['Amount'].map(lambda x:x.replace('$','')), errors='coerce')
    df['Is Fraud?'] = df['Is Fraud?']=='Yes'

    df.drop(['Year','Month','Day','Time'],axis=1,inplace=True)
    df.rename(columns=dict(zip(origin,names)), inplace=True)

    return df

OLTP vs OLAP 
OLTP : On-Line Transactional Processing --> Bases de datos relacionales (la que da soporte a las aplicaciones)--> CRUD 
OLAP : On-Line Analytical Processing --> Cubos de información (dimensiones/hechos, sirve para análisis) --> SELECT 

## Limpieza total

In [21]:
df = pd.concat(map(validar_datos,archivos),ignore_index=True)
df.shape

(2000000, 9)

In [23]:
df.to_pickle('df_ibm.pkl')

In [3]:
df = pd.read_pickle('df_ibm.pkl')

## Conexión a la base de datos

In [36]:
creds = json.load(open('creds.json'))

In [41]:
cnx = create_engine(f"mysql+pymysql://{creds['user']}:{creds['password']}@{creds['host']}/{creds['database']}").connect()
cnx.closed

False

## Modelo Relacional (OLTP)

### Entidad Usuario 

In [44]:
tbl_user = df[['id_user']].drop_duplicates().reset_index(drop=True)
tbl_user['uuid'] = tbl_user['id_user'].map(lambda x:uuid4())
tbl_user.to_sql('tbl_user',
                cnx,
                if_exists='replace',
                index=False,dtype={'id_user':INTEGER,'uuid':CHAR(36)})

189

### Entidad tarjeta 

In [45]:
tbl_card = df[['id_card','id_user']].drop_duplicates().reset_index(drop=True)
tbl_card['uuid'] = tbl_card['id_card'].map(lambda x:uuid4())
tbl_card = tbl_card.merge(tbl_user.rename(columns={'uuid':'uuid_user'}), on='id_user', how='inner').drop(['id_user','id_card'],axis=1)
tbl_card.to_sql('tbl_card',
                cnx,
                if_exists='replace',
                index=False,dtype={'uuid_user':CHAR(36),'uuid':CHAR(36)})

546

### Entidad tipo txn

In [51]:
tbl_txn_type = df[['d_use_chip']].drop_duplicates().reset_index(drop=True)
tbl_txn_type['uuid'] = tbl_txn_type['d_use_chip'].map(lambda x:uuid4())
tbl_txn_type = tbl_txn_type.rename(columns={'d_use_chip':'txn_type'})
tbl_txn_type.to_sql('tbl_txn_type',
                    cnx,
                    if_exists='replace',
                    index=False,dtype={'txn_type':VARCHAR(18),'uuid':CHAR(36)})

3

In [46]:
df.head(2)

,id_user,id_card,c_amt,d_use_chip,d_merchant_state,id_mcc,d_errors,b_fraud,dt_timestamp
0,1584,3,6.93,Chip Transaction,NY,5921,NaN,False,2017-03-03 10:30:00
1,1584,3,6.38,Chip Transaction,NY,5921,NaN,False,2017-03-04 10:30:00


### Entidad Estado

In [61]:
tbl_state = df[['d_merchant_state']].fillna('OTHER').astype(str).drop_duplicates().reset_index(drop=True)
tbl_state['uuid'] = tbl_state['d_merchant_state'].map(lambda x:uuid4())
tbl_state = tbl_state.rename(columns={'d_merchant_state':'merchant_state'})
tbl_state.to_sql('tbl_state',
                 cnx,
                 if_exists='replace',
                 index=False,dtype={'merchant_state':VARCHAR(24),'uuid':CHAR(36)})

144

### Entidad MCC

In [92]:
mcc = pd.read_excel('lista_mcc.xlsx',skiprows=1)
mcc.columns = ['id_mcc','mcc_description']
mcc['id_mcc'] = pd.to_numeric(mcc['id_mcc'], errors='coerce').fillna(9999).astype(int)
mcc['id_mcc'] = mcc['id_mcc'].map(int).map(lambda x:f'{x:04d}')
tbl_mcc = df[['id_mcc']].drop_duplicates().reset_index(drop=True)
tbl_mcc['id_mcc'] = tbl_mcc['id_mcc'].map(int).map(lambda x:f'{x:04d}')
tbl_mcc = tbl_mcc.merge(mcc, on='id_mcc', how='inner')
tbl_mcc['uuid'] = tbl_mcc['id_mcc'].map(lambda x:uuid4())


In [96]:
tbl_mcc.to_sql('tbl_mcc',
               cnx,
               if_exists='replace',
               index=False,dtype={'id_mcc':CHAR(4),'mcc_description':VARCHAR(200),'uuid':CHAR(36)})

77

In [74]:
mcc['id_mcc'].map(len).value_counts()

id_mcc
4    5
Name: count, dtype: int64

In [66]:
mcc

,id_mcc,mcc_description
0,0742,Veterinary services
1,0743,Wine producers
2,0744,Champagne producers
3,0763,Agricultural Cooperatives
4,0780,Landscaping and horticultural services
